In [1]:
import sys
from pathlib import Path 

sys.path.append(str(Path().resolve().parent.parent))

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "water-quality-prediction"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [3]:
import pandas as pd
from dataretrieval import nwis

def check_tram(site_id):
    print(f"--- Đang kiểm tra trạm {site_id} ---")
    parameter_codes = ["00060", "00010", "00400", "00300", "00095", "63680"]
    for para in parameter_codes:
        try:
            # Tải thử 7 ngày gần nhất
            df = nwis.get_record(sites=site, service='iv', period='P7D', parameterCd=f'{para}') # 00300 là DO
            if df.empty:
                print(f"❌ Trạm {site_id}: Không có dữ liệu {para}")
            else:
                print(f"✅ Trạm {site_id}: Tốt! Có {len(df)} dòng dữ liệu {para}")
        except Exception as e:
            print(f"⚠️ Lỗi khi kiểm tra trạm {site_id}: {e}")

# Test thử danh sách mới
site_ids = [
    "01463500",  # Delaware River at Trenton, NJ
    "14211720",  # Willamette River at Portland, OR
    "03321500",   # Green River at Lock 1 at Spottsville, KY
    "03216070", # Ohio River at Ironton, OH
    "01646500" #Potomac River near Washington, DC
]
for site in site_ids:
    check_tram(site)

--- Đang kiểm tra trạm 01463500 ---
✅ Trạm 01463500: Tốt! Có 668 dòng dữ liệu 00060
✅ Trạm 01463500: Tốt! Có 166 dòng dữ liệu 00010
✅ Trạm 01463500: Tốt! Có 166 dòng dữ liệu 00400
✅ Trạm 01463500: Tốt! Có 166 dòng dữ liệu 00300
✅ Trạm 01463500: Tốt! Có 166 dòng dữ liệu 00095
✅ Trạm 01463500: Tốt! Có 166 dòng dữ liệu 63680
--- Đang kiểm tra trạm 14211720 ---
✅ Trạm 14211720: Tốt! Có 2008 dòng dữ liệu 00060
✅ Trạm 14211720: Tốt! Có 335 dòng dữ liệu 00010
✅ Trạm 14211720: Tốt! Có 335 dòng dữ liệu 00400
✅ Trạm 14211720: Tốt! Có 335 dòng dữ liệu 00300
✅ Trạm 14211720: Tốt! Có 335 dòng dữ liệu 00095
✅ Trạm 14211720: Tốt! Có 335 dòng dữ liệu 63680
--- Đang kiểm tra trạm 03321500 ---
✅ Trạm 03321500: Tốt! Có 655 dòng dữ liệu 00060
✅ Trạm 03321500: Tốt! Có 650 dòng dữ liệu 00010
✅ Trạm 03321500: Tốt! Có 655 dòng dữ liệu 00400
✅ Trạm 03321500: Tốt! Có 654 dòng dữ liệu 00300
✅ Trạm 03321500: Tốt! Có 652 dòng dữ liệu 00095
✅ Trạm 03321500: Tốt! Có 655 dòng dữ liệu 63680
--- Đang kiểm tra trạm 0321

In [3]:
import pandas as pd
from dataretrieval import nwis

site_ids = [
    "01463500",  # Delaware River at Trenton, NJ
    "14211720",  # Willamette River at Portland, OR
    "03321500",   # Green River at Lock 1 at Spottsville, KY
    "03216070", # Ohio River at Ironton, OH
    "01646500" #Potomac River near Washington, DC
]

# 00060: Lưu lượng (Flow)
# 00010: Nhiệt độ (Temp)
# 00400: pH
# 00300: Oxy hòa tan (DO)
# 63680: Độ đục (Turbidity)
# 00095: Độ dẫn điện (EC) - Tham số phụ rất tốt để điền khuyết
parameter_codes = ["00060", "00010", "00400", "00300", "63680", "00095"]


start_date = "2021-01-01"
end_date = "2025-12-31"

print(f"DOWNLOAD DATA FROM {len(site_ids)} STATION: {site_ids}...")

# List chứa dữ liệu các trạm
data_frames = []

# Tải từng trạm (An toàn hơn tải gộp)
for site in site_ids:
    print(f"--- STATION IN PROGRESS {site} ---")
    try:
        # Gọi API tải dữ liệu tức thời (iv)
        # Lưu ý: Hàm trả về (DataFrame, Metadata), ta chỉ lấy phần tử [0]
        data = nwis.get_record(sites=site, service='iv', 
                               start=start_date, end=end_date, 
                               parameterCd=parameter_codes)
        
        # Kiểm tra kiểu dữ liệu trả về để lấy DataFrame chuẩn
        df_temp = data
        
        if not df_temp.empty:
            # Reset index để đưa cột datetime ra ngoài
            df_temp = df_temp.reset_index()
            df_temp['site_no'] = site  # Gán nhãn tên trạm
            data_frames.append(df_temp)
            print(f"SUCCESS!! get {len(df_temp)} records.")
        else:
            print(f"⚠️ Station {site} has no response!")
            
    except Exception as e:
        print(f"❌ Error at {site}: {e}")



DOWNLOAD DATA FROM 5 STATION: ['01463500', '14211720', '03321500', '03216070', '01646500']...
--- STATION IN PROGRESS 01463500 ---
SUCCESS!! get 170436 records.
--- STATION IN PROGRESS 14211720 ---
SUCCESS!! get 526665 records.
--- STATION IN PROGRESS 03321500 ---
SUCCESS!! get 172812 records.
--- STATION IN PROGRESS 03216070 ---
SUCCESS!! get 171822 records.
--- STATION IN PROGRESS 01646500 ---
SUCCESS!! get 211451 records.


In [14]:
for i, df in enumerate(data_frames):
    print(f"Dataframe số {i}: {list(df.columns)}")
   

Dataframe số 0: ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
Dataframe số 1: ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
Dataframe số 2: ['datetime', 'site_no', '00010_2', '00010_2_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
Dataframe số 3: ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
Dataframe số 4: ['datetime', 'site_no', '00010_from multiparameter sonde', '00010_from multiparameter sonde_cd', '00060', '00060_cd', '00095_from multiparameter sonde', '00095_from multiparameter sonde_cd', '00300_from multiparameter sonde', '00300_from multiparameter sonde_cd', '00400_from multiparameter sonde', '00400_from multi

In [22]:
target_cols = ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
for i,df in enumerate(data_frames):
    if len(df.columns) == len(target_cols):
        df.columns = target_cols
        print(f"Đã đổi tên cột cho Dataframe {i}")
        

Đã đổi tên cột cho Dataframe 0
Đã đổi tên cột cho Dataframe 1
Đã đổi tên cột cho Dataframe 2
Đã đổi tên cột cho Dataframe 3
Đã đổi tên cột cho Dataframe 4


In [23]:
for i, df in enumerate(data_frames):
    print(f"Dataframe số {i}: {list(df.columns)}")

Dataframe số 0: ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
Dataframe số 1: ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
Dataframe số 2: ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
Dataframe số 3: ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']
Dataframe số 4: ['datetime', 'site_no', '00010', '00010_cd', '00060', '00060_cd', '00095', '00095_cd', '00300', '00300_cd', '00400', '00400_cd', '63680', '63680_cd']


In [24]:
data_frames[2]

,datetime,site_no,00010,00010_cd,00060,00060_cd,00095,00095_cd,00300,00300_cd,00400,00400_cd,63680,63680_cd
0,2021-01-01 06:00:00+00:00,03321500,5.6,A,23100.0,A,311.0,A,12.9,A,7.8,A,57.9,A
1,2021-01-01 06:15:00+00:00,03321500,5.6,A,23100.0,A,311.0,A,13.0,A,7.8,A,67.3,A
2,2021-01-01 06:30:00+00:00,03321500,5.6,A,23200.0,A,310.0,A,12.9,A,7.8,A,66.5,A
3,2021-01-01 06:45:00+00:00,03321500,5.6,A,23200.0,A,310.0,A,12.9,A,7.8,A,61.5,A
4,2021-01-01 07:00:00+00:00,03321500,5.6,A,23300.0,A,310.0,A,12.9,A,7.8,A,61.1,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172807,2026-01-01 04:45:00+00:00,03321500,NaN,NaN,5340.0,P,326.0,P,11.2,P,7.9,P,23.3,P
172808,2026-01-01 05:00:00+00:00,03321500,8.2,P,5800.0,P,327.0,P,11.3,P,7.9,P,23.3,P
172809,2026-01-01 05:15:00+00:00,03321500,8.2,P,5800.0,P,327.0,P,11.3,P,7.9,P,22.7,P
172810,2026-01-01 05:30:00+00:00,03321500,8.2,P,5800.0,P,327.0,P,11.3,P,7.9,P,22.8,P


In [25]:
concat_df = pd.concat(data_frames,axis=0,ignore_index=True)

In [26]:
rename_map = {
    '00060': 'Flow',
    '00010': 'Temp',
    '00400': 'pH',
    '00300': 'DO',
    '63680': 'Turbidity',
    '00095': 'EC',
    'datetime': 'Time'
}
concat_df.rename(columns=rename_map, inplace=True)

# Lọc bỏ các cột code rác (_cd)
cols = [c for c in concat_df.columns if not c.endswith('_cd')]
final_df = concat_df[cols]
final_df['Time'] = pd.to_datetime(final_df['Time'])
final_df = final_df[final_df['Time'].dt.minute==0]


C:\Users\Admin\AppData\Local\Temp\ipykernel_33584\1724993458.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Time'] = pd.to_datetime(final_df['Time'])


In [27]:
final_df.columns

Index(['Time', 'site_no', 'Temp', 'Flow', 'EC', 'DO', 'pH', 'Turbidity'], dtype='object')

In [ ]:
final_df

In [29]:
from src.Utils.path import DATA_DIR

In [30]:
final_df.to_csv(DATA_DIR/'New_data/USGS/water_data_2021_2025_raw.csv',index=False)